In [7]:
#importar datos a una lista
import pandas as pd
import nltk
from tqdm import tqdm, tqdm_notebook

## Analisis (hay que expandir)

In [8]:
df = pd.read_csv("./data/quora/train.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
print("Numero total de preguntas: {total}".format(total = len(df)))
print("Numero de preguntas repetidas: {b}, ({a:.2f}%)"
      .format(b=df["is_duplicate"].sum(),a=df["is_duplicate"].mean()*100) )
print("Valores nulos: \n{}".format(df.isnull().sum()))

#Si hubiera campos vacios (habia 2) rellenamos con empty
df = df.fillna("empty")

Numero total de preguntas: 404290
Numero de preguntas repetidas: 149263, (36.92%)
Valores nulos: 
id              0
qid1            0
qid2            0
question1       0
question2       2
is_duplicate    0
dtype: int64


## Embeddings

In [10]:
#Registramos pandas en tqdm para usar progress_apply y progress_map en vez de los propios de pandas
tqdm.pandas(tqdm_notebook)
new_df = df.copy()

In [10]:
new_df['question1'] = df.progress_apply(lambda row: nltk.word_tokenize(row['question1']), axis=1)
new_df['question2'] = df.progress_apply(lambda row: nltk.word_tokenize(row['question2']), axis=1)
new_df["avgQ1Q2"] = -1.0

100it [00:00, 1940.56it/s]           
100it [00:00, 1979.84it/s]           


In [11]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,avgQ1Q2
0,0,1,2,"[What, is, the, step, by, step, guide, to, inv...","[What, is, the, step, by, step, guide, to, inv...",0,-1.0
1,1,3,4,"[What, is, the, story, of, Kohinoor, (, Koh-i-...","[What, would, happen, if, the, Indian, governm...",0,-1.0
2,2,5,6,"[How, can, I, increase, the, speed, of, my, in...","[How, can, Internet, speed, be, increased, by,...",0,-1.0
3,3,7,8,"[Why, am, I, mentally, very, lonely, ?, How, c...","[Find, the, remainder, when, [, math, ], 23^, ...",0,-1.0
4,4,9,10,"[Which, one, dissolve, in, water, quikly, suga...","[Which, fish, would, survive, in, salt, water, ?]",0,-1.0



## Codigo no util, guardado por si lo uso en el informe final
#guardamos las preguntas por separados
df = pd.read_csv("./data/quora/train.csv", nrows=100)
df = df.where((pd.notnull(df)), "Vacio")
#df["tokensQ1"] = df["question1"].apply(nltk.word_tokenize)
#df["tokensQ2"] = df["question2"].apply(nltk.word_tokenize)

#df["question1"].to_csv("./data/quora/trainQ1.csv")
#df["question2"].to_csv("./data/quora/trainQ2.csv")
new_df = pd.DataFrame(columns=["unique_id","id1","id2","q1","q2"])
for row in tqdm(df.itertuples(), total=df.shape[0]):
    new_df["unique_id"] = df["id"]
    new_df["id1"] = df["qid1"]
    new_df["id2"] = df["qid2"]
    new_df["q1"] = df["question1"].apply(nltk.word_tokenize)
    new_df["q2"] = df["question2"].apply(nltk.word_tokenize)
    new_df["avgQ1Q2"] = float(-1)

pd.to_numeric(new_df["avgQ1Q2"])
new_df.head()

In [12]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin.gz', 
                                                        binary=True, limit=50000)
#model.similarity('woman', 'man')
#model['man']
#model.most_similar(positive=['woman', 'king'], negative=['man'])

Using TensorFlow backend.


In [13]:
import numpy as np
import scipy.spatial.distance as distance

def avg_feature_vector(words, num_features, model):
    """ Metodo que saca el vector medio de un cantidad de palabras dadas 
        word: palabras separadas de las que queremos obtener el vector medio
        num_features: El tam del vector que representa a una palabra del modelo usado
    """
    """https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt"""
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in model:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])

    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [15]:
for line in tqdm(new_df.itertuples(), total=new_df.shape[0]):
    sentence_1_avg_vector = avg_feature_vector(new_df["question1"][line[0]], num_features=300, model=model)
    sentence_2_avg_vector = avg_feature_vector(new_df["question2"][line[0]], num_features=300, model=model)
    #new_df["avgQ1Q2"][line[0]] =  1 - distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
    new_df.loc["avgQ1Q2",line[0]] =  1 - distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)

100%|██████████| 100/100 [00:00<00:00, 477.18it/s]


In [16]:
new_df.to_csv("./quora/new_df.csv", index=False, encoding='utf-8')